# Chapter 15. Multiple Linear Regression

In [49]:
#필요한 라이브러리, 함수 모음
import random

def dot(v, w):
    """v_1 * w_1 + ... + v_n * w_n"""
    return sum(v_i * w_i for v_i, w_i in zip(v, w))

def total_sum_of_squares(y):
    """평균을 기준으로 y_i의 변화량을 제곱한 값의 총합"""
    return sum(v**2 for v in de_mean(y))

def de_mean(x):
    """translate x by subtracting its mean (so the result has mean 0)"""
    x_bar = mean(x)
    return [x_i - x_bar for x_i in x]

def mean(x):
    return sum(x) / len(x)

def vector_subtract(v, w):
    """subtracts two vectors componentwise"""
    return [v_i - w_i for v_i, w_i in zip(v,w)]

def in_random_order(data):
    """generator that returns the elements of data in random order"""
    indexes = [i for i, _ in enumerate(data)]  # create a list of indexes
    random.shuffle(indexes)                    # shuffle them
    for i in indexes:                          # return the data in that order
        yield data[i]

def scalar_multiply(c, v):
    return [c * v_i for v_i in v]

def minimize_stochastic(target_fn, gradient_fn, x, y, theta_0, alpha_0=0.01):
    data = list(zip(x, y))
    theta = theta_0                             # initial guess
    alpha = alpha_0                             # initial step size
    min_theta, min_value = None, float("inf")   # the minimum so far
    iterations_with_no_improvement = 0

    # if we ever go 100 iterations with no improvement, stop
    while iterations_with_no_improvement < 100:
        value = sum( target_fn(x_i, y_i, theta) for x_i, y_i in data )

        if value < min_value:
            # if we've found a new minimum, remember it
            # and go back to the original step size
            min_theta, min_value = theta, value
            iterations_with_no_improvement = 0
            alpha = alpha_0
        else:
            # otherwise we're not improving, so try shrinking the step size
            iterations_with_no_improvement += 1
            alpha *= 0.9

        # and take a gradient step for each of the data points
        for x_i, y_i in in_random_order(data):
            gradient_i = gradient_fn(x_i, y_i, theta)
            theta = vector_subtract(theta, scalar_multiply(alpha, gradient_i))

    return min_theta

- 단순 회귀 분석에서 변수 x와 y에 대한 상관관계를 밝혀냈다면, 다중 회귀 분석에서는 모델의 성능을 높이기 위해 추가로 다른 종류의 데이터를 모델에 추가한다. 
- 즉, 단순 회귀 분석이 x라는 한 가지 요인만을 고려했다면, 다중 회귀 분석에서는 x, z, v등의 여러 개의 독립변수를 사용하는 선형 모델을 이용한다.
                y = (β1 * x1) + (β2 * x2) + (β3 * x3) ... + (βn * xn) + α + ε
                
- 여기서는 숫자 데이터가 아닌 것이 들어갈 수도 있다. 단, 그런 것들은 가변수(dummy variable)를 만들어서 처리할 수 있다(ex. Binary values)

- 여기서는 예를 들어, 아래와 같은 선형 모델을 시험해 보고자 한다.
        SNS 이용 시간(분) = α + ( β1 * 친구 수 ) + ( β2 * 근무시간 ) + ( β3 * 박사학위 취득 여부 ) + ε

### 15-1. 모델

- 단순 회귀 분석의 모델:
        y = βx + α + ε
- 다중 회귀 분석의 모델: 입력값 x가 여러 개(k개)이다.
        y = (β1 * x1) + (β2 * x2) + (β3 * x3) ... + (βk * xk) + α + ε

In [16]:
#다중 회귀 분석 모델 식
# 파라미터 벡터는 베타(β), 데이터는 x_i, 독립변수 x
def predict(x_i, beta):
    """각 x_i의 첫 번째 항목은 1이라고 가정한다."""
    return dot(x_i, beta)

# beta = [alpha, beta_1, beta_2, ..., beta_k]
# x_i = [1, x_i1, x_i2, ..., x_ik]

### 15-2. 최소자승법에 대한 몇 가지 추가 가정

- 최소자승법: sum_of_squared_errors를 최소화해주는 알파, 베타값을 찾는 것. 
- 우리가 알고싶은 것은 데이터 전체에서 발생하는 총 오류값이다.
- 그러나 r(오류)의 총합은 서로 상쇄될 수 있으므로, r^2들의 총합을 구한다.
- 미분을 사용하면 최소자승법을 구현할 수 있다.
        def sum_of_squared_errors(alpha, beta, x, y):
            return sum(error(alpha, beta, x_i, y_i)**2 for x_i, y_i in zip(x, y))

1. x의 열은 서로 일차독립해야 한다.
  - 일차독립: 어떤 벡터도 다른 벡터의 선형결합으로 만들어질 수 없다.
  - 만약 이 가정이 성립하지 않는다면, β를 추정할 수 없다.

2. x의 모든 열은 오류(ε)와 아무 상관 관계가 없다. 만약 이 가정이 위배될 경우, 아예 다른 β가 추정된다.

- 독립 변수와 오류 사이에 상관관계가 존재한다면, 최소자승법으로 만들어지는 모델은 편향된 β를 추정해 준다.

### 15-3. 모델 학습하기

- 단순 회귀 분석 모델처럼, 오류를 제곱한 값의 합을 최소화해 주는 β를 찾아야 한다.
- 경사 하강법을 사용하여 이를 찾아보자.

In [48]:
#먼저, 최소화할 오차 함수를 만든다.
def error(x_i, y_i, beta):
    return y_i - predict(x_i, beta)

#그리고 SGD(Stochastic Gradient Descent)를 사용하기 위해 예측값 하나에 대한 오차의 제곱값을 쓴다.
def squared_error(x_i, y_i, beta):
    return error(x_i, y_i, beta) ** 2

In [18]:
#미적분을 다룰 수 있다면 오차를 직접 계산할 수 있다.
def squared_error_gradient(x_i, y_i, beta):
    """i번째 오류 제곱 값의 beta에 대한 기울기"""
    return [-2 * x_ij * error(x_i, y_i, beta)
            for x_ij in x_i]

# def squared_error(x_i, y_i, theta):
#     alpha, beta = theta
#     return error(alpha, beta, x_i, y_i) ** 2

# def squared_error_gradient(x_i, y_i, theta):
#     alpha, beta = theta
#     return [-2 * error(alpha, beta, x_i, y_i),        #알파에 대한 편미분
#             -2 * error(alpha, beta, x_i, y_i) * x_i]  #베타에 대한 편미분

In [32]:
#이제, SGD를 사용해서 최적의 베타를 계산하자.
def estimate_beta(x, y):
    beta_initial = [random.random() for x_i in x[0]]
    return minimize_stochastic(squared_error,
                              squared_error_gradient,
                              x, y,
                              beta_initial,
                              0.001)

In [54]:
#챕터 처음에서 언급한 다중 선형 모델에 대한 데이터.
#SNS 이용 시간(분) = α + ( β1 * 친구 수 ) + ( β2 * 근무시간 ) + ( β3 * 박사학위 취득 여부 ) + ε

x = [[1,49,4,0],[1,41,9,0],[1,40,8,0],[1,25,6,0],[1,21,1,0],[1,21,0,0],[1,19,3,0],
     [1,19,0,0],[1,18,9,0],[1,18,8,0],[1,16,4,0],[1,15,3,0],[1,15,0,0],[1,15,2,0],
     [1,15,7,0],[1,14,0,0],[1,14,1,0],[1,13,1,0],[1,13,7,0],[1,13,4,0],[1,13,2,0],
     [1,12,5,0],[1,12,0,0],[1,11,9,0],[1,10,9,0],[1,10,1,0],[1,10,1,0],[1,10,7,0],
     [1,10,9,0],[1,10,1,0],[1,10,6,0],[1,10,6,0],[1,10,8,0],[1,10,10,0],[1,10,6,0],
     [1,10,0,0],[1,10,5,0],[1,10,3,0],[1,10,4,0],[1,9,9,0],[1,9,9,0],[1,9,0,0],
     [1,9,0,0],[1,9,6,0],[1,9,10,0],[1,9,8,0],[1,9,5,0],[1,9,2,0],[1,9,9,0],[1,9,10,0],
     [1,9,7,0],[1,9,2,0],[1,9,0,0],[1,9,4,0],[1,9,6,0],[1,9,4,0],[1,9,7,0],[1,8,3,0],
     [1,8,2,0],[1,8,4,0],[1,8,9,0],[1,8,2,0],[1,8,3,0],[1,8,5,0],[1,8,8,0],[1,8,0,0],
     [1,8,9,0],[1,8,10,0],[1,8,5,0],[1,8,5,0],[1,7,5,0],[1,7,5,0],[1,7,0,0],[1,7,2,0],
     [1,7,8,0],[1,7,10,0],[1,7,5,0],[1,7,3,0],[1,7,3,0],[1,7,6,0],[1,7,7,0],[1,7,7,0],
     [1,7,9,0],[1,7,3,0],[1,7,8,0],[1,6,4,0],[1,6,6,0],[1,6,4,0],[1,6,9,0],[1,6,0,0],
     [1,6,1,0],[1,6,4,0],[1,6,1,0],[1,6,0,0],[1,6,7,0],[1,6,0,0],[1,6,8,0],[1,6,4,0],
     [1,6,2,1],[1,6,1,1],[1,6,3,1],[1,6,6,1],[1,6,4,1],[1,6,4,1],[1,6,1,1],[1,6,3,1],
     [1,6,4,1],[1,5,1,1],[1,5,9,1],[1,5,4,1],[1,5,6,1],[1,5,4,1],[1,5,4,1],[1,5,10,1],
     [1,5,5,1],[1,5,2,1],[1,5,4,1],[1,5,4,1],[1,5,9,1],[1,5,3,1],[1,5,10,1],[1,5,2,1],
     [1,5,2,1],[1,5,9,1],[1,4,8,1],[1,4,6,1],[1,4,0,1],[1,4,10,1],[1,4,5,1],[1,4,10,1],
     [1,4,9,1],[1,4,1,1],[1,4,4,1],[1,4,4,1],[1,4,0,1],[1,4,3,1],[1,4,1,1],[1,4,3,1],
     [1,4,2,1],[1,4,4,1],[1,4,4,1],[1,4,8,1],[1,4,2,1],[1,4,4,1],[1,3,2,1],[1,3,6,1],
     [1,3,4,1],[1,3,7,1],[1,3,4,1],[1,3,1,1],[1,3,10,1],[1,3,3,1],[1,3,4,1],[1,3,7,1],
     [1,3,5,1],[1,3,6,1],[1,3,1,1],[1,3,6,1],[1,3,10,1],[1,3,2,1],[1,3,4,1],[1,3,2,1],
     [1,3,1,1],[1,3,5,1],[1,2,4,1],[1,2,2,1],[1,2,8,1],[1,2,3,1],[1,2,1,1],[1,2,9,1],
     [1,2,10,1],[1,2,9,1],[1,2,4,1],[1,2,5,1],[1,2,0,1],[1,2,9,1],[1,2,9,1],[1,2,0,1],
     [1,2,1,1],[1,2,1,1],[1,2,4,1],[1,1,0,1],[1,1,2,1],[1,1,2,1],[1,1,5,1],[1,1,3,1],
     [1,1,10,1],[1,1,6,1],[1,1,0,1],[1,1,8,1],[1,1,6,1],[1,1,4,1],[1,1,9,1],[1,1,9,1],
     [1,1,4,1],[1,1,2,1],[1,1,9,1],[1,1,0,1],[1,1,8,1],[1,1,6,1],[1,1,1,1],[1,1,1,1],[1,1,5,1]]
#[1,   #상수항
# 49,  #친구의 수
# 4,   #하루 근무 시간,
# 0,   #박사학위 여부]
daily_minutes_good = [68.77,51.25,52.08,38.36,44.54,57.13,51.4,41.42,31.22,34.76,54.01,38.79,
                      47.59,49.1,27.66,41.03,36.73,48.65,28.12,46.62,35.57,32.98,35,26.07,23.77
                      ,39.73,40.57,31.65,31.21,36.32,20.45,21.93,26.02,27.34,23.49,46.94,30.5,
                      33.8,24.23,21.4,27.94,32.24,40.57,25.07,19.42,22.39,18.42,46.96,23.72,26.41,
                      26.97,36.76,40.32,35.02,29.47,30.2,31,38.11,38.18,36.31,21.03,30.86,36.07,
                      28.66,29.08,37.28,15.28,24.17,22.31,30.17,25.53,19.85,35.37,44.6,17.23,13.47,
                      26.33,35.02,32.09,24.81,19.33,28.77,24.26,31.98,25.73,24.86,16.28,34.51,15.23,
                      39.72,40.8,26.06,35.76,34.76,16.13,44.04,18.03,19.65,32.62,35.59,39.43,14.18,
                      35.24,40.13,41.82,35.45,36.07,43.67,24.61,20.9,21.9,18.79,27.61,27.21,26.61,
                      29.77,20.59,27.53,13.82,33.2,25,33.1,36.65,18.63,14.87,22.2,36.81,25.53,24.62,
                      26.25,18.21,28.08,19.42,29.79,32.8,35.99,28.32,27.79,35.88,29.06,36.28,14.1,
                      36.63,37.49,26.9,18.58,38.48,24.48,18.95,33.55,14.24,29.04,32.51,25.63,22.22,
                      19,32.73,15.16,13.9,27.2,32.01,29.27,33,13.74,20.42,27.32,18.23,35.35,28.48,
                      9.08,24.62,20.12,35.26,19.92,31.02,16.49,12.16,30.7,31.22,34.65,13.13,27.51,
                      33.2,31.57,14.1,33.42,17.44,10.12,24.42,9.82,23.39,30.93,15.03,21.67,31.09,
                      33.29,22.61,26.89,23.48,8.38,27.81,32.35,23.84]

In [40]:
random.seed(0)
beta = estimate_beta(x, daily_minutes_good)
print(beta)

#모델 완성
# 분(minutes) = 30.62 +  ( 0.972 * 친구 수 ) + ( -1.867 * 근무시간 ) + ( 0.916 * 박사학위 취득 여부 )

[30.619881701311712, 0.9702056472470465, -1.8671913880379478, 0.9163711597955347]


### 15-4. 모델 해석하기

- 모델의 계수는 다른 모든 것이 동일할 때 해당 항목의 영향력을 나타낸다.
    * ex. 다른 모든 것이 동일할 때, 친구 수가 한 명 증가하면 사용자가 하루 평균 SNS 사이트에서 보내는 시간은 1분 증가한다.
    * ex. 다른 모든 것이 동일할 때, 근무 시간이 한 시간 증가하면 사용자가 하루 평균 SNS 사이트에서 보내는 시간은 2분 감소하다.
- 그러나, 이러한 해석은 변수 간의 관계를 직접적으로 설명해 주지 못한다(다른 모든 것이 동일하다는 가정).
    * ex. 친구의 수가 다른 사용자들의 근무 시간은 서로 다를 수 있다.
- 변수가 점점 추가되기 시작하면, 각 계수가 유의미한지 살펴봐야 한다. 변수끼리 곱한 값, 변수의 log값, 변수의 제곱값 등 수 많은 새로운 변수를 추가할 수 있기 때문이다.

### 15-5. 적합성(Goodness of fit)

In [47]:
#모델의 r 제곱값을 다시 계산해 보자.

def multiple_r_squared(x, y, beta):
    sum_of_squared_errors = sum(error(x_i, y_i, beta) ** 2
                               for x_i, y_i in zip(x, y))
    return 1.0 - sum_of_squared_errors / total_sum_of_squares(y)

print("r-squared: ", multiple_r_squared(x, daily_minutes_good, beta)) #-.68
#ch.14의: r_squared(alpha, beta, num_friends_good, daily_minutes_good) == 0.329에 비해 증가했음을 알 수 있음.

r-squared:  0.6800074955952597


- 회귀 분석 모델에 새로운 변수를 추가하면, r 제곱값은 어쩔 수 없이 증가한다.
- 단순 회귀 분석 모델은, 공통되는 하나의 변수를 제외한 나머지 변수들의 값이 0인 다중 회귀 분석 모델과 동일하다.
- 즉, 최적의 다중 회귀 분석 모델은 언제나 단순 회귀 분석 모델보다 작은 오류를 갖게 된다.

- 그렇게 때문에, 다중 회귀 분석 모델에서는 각 계수의 "표준 오차"를 확인해야 한다(추정된 β1의 계수가 얼마나 확실한지 알 수 있음).
- 모델 자체는 주어진 데이터에 적합할 수 있지만, 몇몇 독립변수 간에 어떤 상관관게가 있다면 이 변수들의 계수는 무의미할 것이다.

- 오차를 측정하기 위해서는, 각 오류(ε1)는 독립이며 평균은 0이고 표준편차는 σ인 정규분포의 확률변수라는 가정이 필요하다.
- 이러한 가정을 따르면 통계 소프트웨어는 선형대수를 이용하여 각 계수의 표준편차를 계산해 준다.
- 표준 오차가 클수록 해당 계수는 무의미해진다.